Загрузка библиотек.

In [271]:
import pandas as pd
from IPython.display import display, HTML

Загрузка листов excel файла. 

In [272]:
df_articles = pd.read_excel('Аналитик данных _Тестовое задание _РБК.xlsx', sheet_name = 'База публикаций')
df_views = pd.read_excel('Аналитик данных _Тестовое задание _РБК.xlsx', sheet_name = 'Выгрузка', skiprows = lambda x: x in [0, 1])

Просмотр данных, их типов, пропущенных значений.

In [273]:
df_articles.head()

,ID публикации,Заголовок материала,URL
0,5bf52faf9a79477da426e81b,Как обанкротиться подешевле: прагматичные наст...,https://pro.rbc.ru/news/5bf52faf9a79477da426e81b
1,5c04ee439a7947b3fe48fe55,Имитация тунца: как искусственная рыба вытесня...,https://pro.rbc.ru/news/5c04ee439a7947b3fe48fe55
2,5c04f06c9a7947b48d48fe60,Сколько россияне тратят на путешествия,https://pro.rbc.ru/news/5c04f06c9a7947b48d48fe60
3,5c0695d69a79472e0096479e,Самые ходовые: какие специалисты в строительст...,https://pro.rbc.ru/news/5c0695d69a79472e0096479e
4,5c07891f9a794775e61affd1,Как запустить новый продукт: четыре обязательн...,https://pro.rbc.ru/news/5c07891f9a794775e61affd1


In [274]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID публикации        454 non-null    object
 1   Заголовок материала  454 non-null    object
 2   URL                  454 non-null    object
dtypes: object(3)
memory usage: 10.8+ KB


In [275]:
df_articles.isnull().count()

ID публикации          454
Заголовок материала    454
URL                    454
dtype: int64

In [276]:
df_views.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2696 entries, 0 to 2695
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Адрес страницы  2696 non-null   object
 1   Просмотры       2696 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 42.2+ KB


In [277]:
df_views.describe()

,Просмотры
count,2696.000000
mean,106.040430
std,333.579372
min,1.000000
25%,4.000000
50%,11.000000
75%,34.250000
max,2849.000000


In [278]:
df_views.isnull().count()

Адрес страницы    2696
Просмотры         2696
dtype: int64

Типы данных верные, пропущенных значений нет.

Исключение из списка ненужных адресов.

In [279]:
URL = {'test.pro',
'test.v2.pro',
'feature-rbcnews',
'/preview/',
'staging.pro',
'staging.v2.pro'}

In [280]:
for i in URL:
    df_views = df_views.loc[~df_views['Адрес страницы'].str.contains(i)]
df_views

,Адрес страницы,Просмотры
0,https://pro.rbc.ru/news/5c4af9ed9a7947bd94fe0cc9,2849
1,http://pro.rbc.ru/news/5c61b2d39a794798b96fab0c,2845
2,https://pro.rbc.ru/news/5c6195799a79478b57ba6cf3,2819
3,https://pro.rbc.ru//news/5c63fbff9a7947481bf4ce1b,2736
4,https://pro.rbc.ru/news/5c46d39b9a7947fc20fbcce6,2704
...,...,...
2691,https://pro.rbc.ru/news/5c63fbff9a7947481bf4ce1b,1
2692,https://pro.rbc.ru/news/5c6297279a7947d83339d513,1
2693,https://pro.rbc.ru/news/5c62c45c9a7947eaedb915e4,1
2694,https://pro.rbc.ru/news/5c643a319a794760bbe62974,1


Группировка по адресам ссылок и подсчет общего количества просмотров, сортировка по убыванию просмотров.

In [281]:
df_views = df_views.pivot_table(
    values = 'Просмотры'
    , index = 'Адрес страницы'
    , aggfunc = 'sum'
).sort_values('Просмотры', ascending = False)
df_views

,Просмотры
Адрес страницы,
https://pro.rbc.ru/news/5c658e479a79474d32847233,11320
https://pro.rbc.ru/news/5c65950d9a79474f82f780d7,7422
https://pro.rbc.ru/news/5c503adb9a794784a693903b,7044
https://pro.rbc.ru/news/5c6454df9a794769a732e049,6483
https://pro.rbc.ru/news/5c643a319a794760bbe62974,6325
...,...
https://pro.rbc.ru/rbcfreenews/5c61a5d59a7947953f607f2f,1
https://pro.rbc.ru//news/5bf584f29a794719f9ae353d,1
https://pro.rbc.ru/news/5b28f5269a794763036e17e9,1


Соединение двух таблиц для получения общего списка с заголовками, адресами и просмотрами.

In [282]:
df_merge = df_views.merge(df_articles, left_on = 'Адрес страницы', right_on = 'URL')\
            [['Заголовок материала', 'URL', 'Просмотры']]
df_merge

,Заголовок материала,URL,Просмотры
0,"Глава BlaBlaCar — РБК: «Мы оказывали услуги, н...",https://pro.rbc.ru/news/5c658e479a79474d32847233,11320
1,«Алые паруса» сворачиваются: почему владельцы ...,https://pro.rbc.ru/news/5c65950d9a79474f82f780d7,7422
2,Недооцененный экспорт: что имеет смысл продава...,https://pro.rbc.ru/news/5c503adb9a794784a693903b,7044
3,Требования работодателя vs ожидания кандидатов...,https://pro.rbc.ru/news/5c6454df9a794769a732e049,6483
4,Восемь способов увеличить средний чек,https://pro.rbc.ru/news/5c643a319a794760bbe62974,6325
...,...,...,...
369,Минпромторг: в 2018 году производство строймат...,https://pro.rbc.ru/news/5c5055d39a79478fbfa98466,2
370,Какие неудобства ждут клиентов самозанятых гра...,https://pro.rbc.ru/news/5c4185729a79472af5fd3e5d,2
371,Бананы и киндер-сюрпризы: что москвичи заказыв...,https://pro.rbc.ru/news/5c125b1a9a794773c945fd60,2
372,Ставка на zero: как девелоперы расплачиваются ...,https://pro.rbc.ru/news/5c0fcd779a7947a6b2f354f7,1


Добавление столбца с кликабельными ссылками.  
Сохранение 20-ти самых посещаемых. 

In [283]:
def make_clickable(URL, name):
    return f'<a href="{URL}" target="_blank">{name}</a>'
df_merge['ТОП 20'] = df_merge.apply(lambda x: make_clickable(x['URL'], x['Заголовок материала']), axis = 1)
df_merge[['ТОП 20']].head(20).style.hide(axis="index").format({'url': make_clickable})

ТОП 20
"Глава BlaBlaCar — РБК: «Мы оказывали услуги, но ничего не зарабатывали»"
«Алые паруса» сворачиваются: почему владельцы сети выходят из бизнеса
Недооцененный экспорт: что имеет смысл продавать за рубеж
Требования работодателя vs ожидания кандидатов в ТЭКе — 2019
Восемь способов увеличить средний чек
Менеджеры РФ получают дополнительное образование в основном за свой счет
Как трансформируется логистика в ретейле: лучшие практики
Эксперты: российскому рынку кофеен можно «поправиться» на 870 млрд руб.
Математика российского люкса: точный портрет покупателя предметов роскоши
Россиянам все труднее платить ипотеку и покупать жилье


In [284]:
from IPython import display
display.Image("https://pro.rbc.ru/news/5c658e479a79474d32847233")

ValueError: Cannot embed the 'https://pro.rbc.ru/news/5c658e479a79474d32847233' image format